### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active 

In [4]:
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone drove Mary to school")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'drove',
   'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
 'words': ['Someone', 'drove', 'Mary', 'to', 'school']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'drove',
     'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
   'words': ['Someone', 'drove', 'Mary', 'to', 'school']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Ralph'}, {'first_name': 'Linda'}, {'first_name': 'Susan'}, {'first_name': 'Mary'}, {'first_name': 'Ralph'}, {'first_name': 'Tim'}, {'first_name': 'Melissa'}, {'first_name': 'Sam'}, {'first_name': 'Anthony'}, {'first_name': 'Roger'}, {'first_name': 'Nancy'}, {'first_name': 'James'}, {'first_name': 'Margaret'}, {'first_name': 'Louis'}, {'first_name': 'Francis'}, {'first_name': 'Lisa'}, {'first_name': 'Ann'}, {'first_name': 'Larry'}, {'first_name': 'Kim'}, {'first_name': 'Margaret'}, {'first_name': 'Roger'}, {'first_name': 'Ed'}, {'first_name': 'Suzanne'}, {'first_name': 'Melissa'}, {'first_name': 'Alexandra'}, {'first_name': 'Andrew'}, {'first_name': 'Betty'}, {'first_name': 'Nicole'}, {'first_name': 'William'}, {'first_name': 'Laura'}, {'first_name': 'Harriet'}, {'first_name': 'Matthew'}, {'first_name': 'Harriet'}, {'first_name': 'Maria'}, {'first_name': 'Jason'}, {'first_name': 'Rose'}, {'first_name': 'Bruce'}, {'first_name': 'Keith'}, {'first_name': 'Fiona'}, {'f

In [12]:
with open('Dataset_BERT/Active.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Active_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Ralph', 'drove', 'John', 'to', 'school', '.'] [ True]
['Linda', 'drove', 'John', 'to', 'school', '.'] [ True]
['Susan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Mary', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ralph', 'drove', 'John', 'to', 'school', '.'] [ True]
['Tim', 'drove', 'John', 'to', 'school', '.'] [ True]
['Melissa', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sam', 'drove', 'John', 'to', 'school', '.'] [ True]
['Anthony', 'drove', 'John', 'to', 'school', '.'] [ True]
['Roger', 'drove', 'John', 'to', 'school', '.'] [ True]
['Nancy', 'drove', 'John', 'to', 'school', '.'] [ True]
['James', 'drove', 'John', 'to', 'school', '.'] [ True]
['Margaret', 'drove', 'John', 'to', 'school', '.'] [ True]
['Louis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Francis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Lisa', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ann', 'drove', 'John', 'to

['Frances', 'drove', 'John', 'to', 'school', '.'] [ True]
['Hugh', 'drove', 'John', 'to', 'school', '.'] [ True]
['Eric', 'drove', 'John', 'to', 'school', '.'] [ True]
['Frank', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alexandra', 'drove', 'John', 'to', 'school', '.'] [ True]
['Caroline', 'drove', 'John', 'to', 'school', '.'] [ True]
['Angela', 'drove', 'John', 'to', 'school', '.'] [ True]
['Eleanor', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jay', 'drove', 'John', 'to', 'school', '.'] [ True]
['Harold', 'drove', 'John', 'to', 'school', '.'] [ True]
['John', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jason', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sandra', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alfred', 'drove', 'John', 'to', 'school', '.'] [ True]
['Louis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Benjamin', 'drove', 'John', 'to', 'school', '.'] [ True]
['Rachel', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alan', 'drove', 'John', 'to', 's

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Active_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb

In [16]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']#second verb
        
    return predicate_structure

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [20]:
with open('Dataset_BERT/Passive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/Passive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)


In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_Passive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 